In [ ]:
from init_notebook import *

In [ ]:
from experiments.datasets import *

In [ ]:
next(iter(TeletextIterableDataset()))

# get used characters 

In [ ]:
character_counts = {}
try:
    for text, meta in tqdm(TeletextIterableDataset()):
        for c in text:
            character_counts[c] = character_counts.get(c, 0) + 1
except KeyboardInterrupt:
    pass
len(character_counts)

In [ ]:
from src.util.text_encoder import TextEncoder
text_encoder = TextEncoder("bytefreq")

def embed_text(text: str):
    return text_encoder.encode([text])[0].numpy().astype(np.float16)

display(px.bar(embed_text("Hello!")))

In [ ]:
NUM_FEATURES = 10_000
embeddings = []
try:
    for text, meta in tqdm(
            TeletextIterableDataset()
            .shuffle(100_000)
            .limit(NUM_FEATURES)
    ):
        embeddings.append(embed_text(text))
        
except KeyboardInterrupt:
    pass
    
embeddings = np.stack(embeddings)
embeddings.shape

In [ ]:
px.bar(embeddings.mean(0))

In [ ]:
NUM_CLUSTERS = 10

from sklearn.cluster import KMeans, SpectralClustering

#clusterer = SpectralClustering(n_clusters=10)
clusterer = KMeans(n_clusters=NUM_CLUSTERS, n_init="auto")

cluster_labels = clusterer.fit_predict(embeddings)

In [ ]:
px.bar(np.histogram(cluster_labels, clusterer.n_clusters, (0, clusterer.n_clusters))[0])

In [ ]:
num_shown = {}
try:
    for text, meta in tqdm(
            TeletextIterableDataset()
            # .shuffle(100_000)
            .limit(NUM_FEATURES)
    ):
        embed = embed_text(text)[None, :]
        l = clusterer.predict(embed)[0]
        num_shown[l] = num_shown.get(l, 0) + 1
        if num_shown[l] < 2:
            print(f"----- {l} ----")
            print(text)
            
except KeyboardInterrupt:
    pass
    

In [ ]:
np.save?